In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../preprocess/1125_all.csv')
df = pd.DataFrame(data)
df = df.dropna()
# df.rename(columns={"160":"target"}, inplace=True)
data = df.iloc[:, :-1]
target = df['target']


In [21]:
from tensorflow import keras
import tensorflow.keras.backend as K

K.clear_session()
model = keras.Sequential()
model.add(keras.layers.LSTM(15, activation="tanh", input_shape=(train_input.shape[1],1), return_sequences=True))
model.add(keras.layers.LSTM(7, activation="tanh", dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 160, 15)           1020      
                                                                 
 lstm_1 (LSTM)               (None, 7)                 644       
                                                                 
 dense (Dense)               (None, 1)                 8         
                                                                 
Total params: 1,672
Trainable params: 1,672
Non-trainable params: 0
_________________________________________________________________


In [22]:
repeats = []

for i in range(10):
    train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, stratify=target, random_state=0)
    train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, stratify=train_target, random_state=0)

    model.compile(optimizer='rmsprop', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

    checkpoint_cb = keras.callbacks.ModelCheckpoint('test.h5')
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights=True)

    history = model.fit(train_input, train_target, 
                        epochs=10, batch_size = 50,
                        validation_data = (val_input, val_target),
                        callbacks=[checkpoint_cb, early_stopping_cb])

    val_evaluate = model.evaluate(val_input, val_target)
    test_evaluate = model.evaluate(test_input, test_target)
    
    repeat = []
    repeat.append(history.history['loss'][-1])
    repeat.append(history.history['accuracy'][-1])
    repeat.append(history.history['val_loss'][-1])
    repeat.append(history.history['val_accuracy'][-1])
    repeat.append(val_evaluate[0])
    repeat.append(val_evaluate[1])
    repeat.append(test_evaluate[0])
    repeat.append(test_evaluate[0])
    repeats.append(repeat)


print(repeats)

Epoch 1/10
64/64 [==============================] - 5s 29ms/step - loss: 0.6303 - accuracy: 0.6495 - val_loss: 0.5634 - val_accuracy: 0.6713
Epoch 2/10
64/64 [==============================] - 1s 13ms/step - loss: 0.5181 - accuracy: 0.7836 - val_loss: 0.4782 - val_accuracy: 0.8269
Epoch 3/10
64/64 [==============================] - 1s 13ms/step - loss: 0.4619 - accuracy: 0.8244 - val_loss: 0.4231 - val_accuracy: 0.8457
Epoch 4/10
64/64 [==============================] - 1s 13ms/step - loss: 0.4180 - accuracy: 0.8458 - val_loss: 0.4182 - val_accuracy: 0.8407
Epoch 5/10
64/64 [==============================] - 1s 13ms/step - loss: 0.4043 - accuracy: 0.8502 - val_loss: 0.4257 - val_accuracy: 0.8356
Epoch 6/10
64/64 [==============================] - 1s 14ms/step - loss: 0.3949 - accuracy: 0.8536 - val_loss: 0.4698 - val_accuracy: 0.8356
Epoch 7/10
64/64 [==============================] - 1s 14ms/step - loss: 0.3880 - accuracy: 0.8577 - val_loss: 0.3712 - val_accuracy: 0.8570
Epoch 8/10
64

In [23]:
col_name = ['loss', 'acc', 'val_loss', 'val_acc', 'val_eval_loss', 'val_eval_acc', 'test_eval_loss', 'test_eval_acc']

result = pd.DataFrame(repeats, columns = col_name)
result

,loss,acc,val_loss,val_acc,val_eval_loss,val_eval_acc,test_eval_loss,test_eval_acc
0,0.371790,0.862751,0.349228,0.864492,0.349228,0.864492,0.332913,0.332913
1,0.353115,0.869347,0.328184,0.882058,0.328184,0.882058,0.337494,0.337494
2,0.340998,0.875942,0.369345,0.858218,0.369345,0.858218,0.325136,0.325136
3,0.331742,0.879711,0.323833,0.878294,0.323833,0.878294,0.305515,0.305515
4,0.325940,0.884108,0.293632,0.889586,0.293632,0.889586,0.292249,0.292249
5,0.329490,0.880339,0.327260,0.869511,0.327260,0.869511,0.305333,0.305333
6,0.319931,0.883480,0.286619,0.893350,0.286619,0.893350,0.289508,0.289508
7,0.324465,0.883794,0.318460,0.880803,0.318460,0.880803,0.309726,0.309726
8,0.353097,0.865264,0.389895,0.835634,0.389895,0.835634,0.352396,0.352396
9,0.322134,0.877827,0.303632,0.890841,0.303632,0.890841,0.302386,0.302386
